# Mapzen python code

You will need to
- install Enthought or Anaconda, or another reader
- install pandas, requests and json library

## Always execute these next two cells of code!

This is a MapZen key - you will need to use your own. Please note that you should NEVER share your key to GitHub or elsewhere.

In [1]:
key = 'mapzen-yourkey'
endpoint = "https://matrix.mapzen.com/many_to_many"
 

In [2]:
import requests
import json
import pandas

## Loading a keyfile

I've specificed a keyFile which stores my MapZen key so I can update this code without having to delete my key each time! And so can you. Create a json file in a text editor:

{
    
    "key":"your-key-here"
    
}

And save it as key.json somewhere not in your git directory. Then modify the below folder and it should update your key!

In [3]:
keyFolder = "/Users/martin/Dropbox/CASA/research/2015-08 EPSRC Freight Traffic Control/misc code/mapzen key/"
keyFile = "key.json"
with open(keyFolder+keyFile) as data_file:    
    keyJSON = json.load(data_file)
key = keyJSON['key']

# 1. Example query:

In [20]:
path=json.dumps({"locations":[{"lat":40.744014,"lon":-73.990508},{"lat":40.739735,"lon":-73.979713},{"lat":40.752522,"lon":-73.985015},{"lat":40.750117,"lon":-73.983704},{"lat":40.750552,"lon":-73.993519}], "costing":"auto"})

In [21]:
#params = {"json": json.dumps(route), "api_key": key}
params = {"json":path,"api_key":key}
req = requests.get(endpoint, params=params)

In [22]:
print req.text

{"locations":[[{"lon":-73.990509,"lat":40.744015},{"lon":-73.979713,"lat":40.739735},{"lon":-73.985016,"lat":40.752522},{"lon":-73.983704,"lat":40.750118},{"lon":-73.993523,"lat":40.750553}]],"units":"km","many_to_many":[[{"distance":0.000,"time":0,"to_index":0,"from_index":0},{"distance":1.346,"time":335,"to_index":1,"from_index":0},{"distance":1.944,"time":341,"to_index":2,"from_index":0},{"distance":1.730,"time":340,"to_index":3,"from_index":0},{"distance":1.202,"time":210,"to_index":4,"from_index":0}],[{"distance":2.047,"time":362,"to_index":0,"from_index":1},{"distance":0.000,"time":0,"to_index":1,"from_index":1},{"distance":2.270,"time":371,"to_index":2,"from_index":1},{"distance":1.903,"time":287,"to_index":3,"from_index":1},{"distance":2.303,"time":390,"to_index":4,"from_index":1}],[{"distance":1.841,"time":348,"to_index":0,"from_index":2},{"distance":2.738,"time":346,"to_index":1,"from_index":2},{"distance":0.000,"time":0,"to_index":2,"from_index":2},{"distance":0.761,"time":2

# 2. Working with a small file (fewer than 50 locations)
## Specify the filename and put it in the "data" folder

Pay attention to the format of the file...

In [40]:
#filename = "Example1.csv"
filename = "Example3_fake.csv"

In [41]:
dataIn = pandas.read_csv("data/"+filename)
dataIn.head()

,Node ID,Lat,Lon,Unnamed: 3
0,1,51.500000,-0.100000,NaN
1,2,51.498926,-0.102354,NaN
2,3,51.499040,-0.102451,NaN
3,4,51.503091,-0.099655,NaN
4,5,51.493588,-0.092621,NaN


## Create dict for locations

In [42]:
locations = []
for i in range(len(dataIn.index)):
    locations.append({"lon":dataIn.Lon.iloc[i], "lat":dataIn.Lat.iloc[i]})

test1 = locations

## Specify the travel mode
- auto (car)
- pedestrian
- bicyle

In [43]:
costing = "auto"

## Create a new dict with this and other prefs and feed it into a new json object

In [44]:
dictPath = {"locations":locations, 
            "costing":costing}
#print dictPath

In [45]:
path = json.dumps(dictPath)

## Send a request to MapZen

In [46]:
params = {"json":path,"api_key":key}
req = requests.get(endpoint, params=params)

In [47]:
#print req.url

In [48]:
jsonback = req.json()
locationsOut = jsonback.get("locations")
rawMatrix = jsonback.get("many_to_many")

## Specify Matrix

Create a pandas dataframe, which is easy to save to csv. Here we use "distance", but we could also use "time".

Here the **origin** is the **column**, and the **destination** is the **row**. The matrix is not necessarily symmetric.

In [49]:
cols = list(range(1, 51))
mo = pandas.DataFrame(columns=cols, index = cols)

In [50]:
for row in rawMatrix:
    for edge in row:
        D = edge['to_index']
        O = edge['from_index']
        distance = edge['distance']
        mo.iloc[D,O]= distance
mo.head()

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
1,0,0.32,0.325,0.64,1.664,2.322,2.192,3.288,4.225,2.79,...,7.092,7.508,7.509,7.756,6.687,6.457,9.076,8.406,8.017,9.275
2,0.32,0,0.044,0.945,1.452,2.11,1.98,3.076,4.013,2.578,...,6.88,7.296,7.297,7.544,6.475,6.245,8.864,8.194,7.805,9.063
3,0.335,0.044,0,0.94,1.457,2.115,1.985,3.081,4.018,2.583,...,6.885,7.301,7.302,7.549,6.48,6.25,8.869,8.199,7.81,9.068
4,0.575,0.831,0.836,0,1.853,2.511,2.288,2.913,3.85,2.927,...,6.717,7.133,7.134,7.381,6.312,6.082,8.701,8.031,7.642,8.9
5,2.765,2.445,2.44,3.098,0,0.821,1.27,1.895,2.832,1.909,...,5.699,6.495,6.496,6.363,5.294,5.064,7.683,7.013,6.624,7.882


In [51]:
mo.to_csv("dataOut/"+filename)

# 3. Larger files (more than 50 locations)

In [59]:
filename = "Example4_fake"

In [60]:
dataIn = pandas.read_csv("data/"+filename+".csv")
dataIn.head()

,Node ID,Lat,Lon,Unnamed: 3
0,1,51.500000,-0.100000,NaN
1,2,51.442291,-0.012729,NaN
2,3,51.342476,-0.066382,NaN
3,4,51.290540,-0.060338,NaN
4,5,51.221005,-0.005881,NaN


## Split this into multiple location lists (a list of lists)

Make sure your column names match up here

In [61]:
breakup = 1 + int(len(dataIn)/50)
print "breakup"
print breakup
print type(breakup)

locations = []
for i in range(0, breakup):
    locations.append([])
    
#latcol = "Delivery Lat"
#loncol = 'Deliver Long'

latcol = "Lat"
loncol = 'Lon'

n=0
for i in range(len(dataIn.index)):
    n = i/50
    locations[n].append({"lon":dataIn[loncol].iloc[i], "lat":dataIn[latcol].iloc[i]})

breakup
2
<type 'int'>


## Save into one .csv

Called "FilenameIn_out.csv"

In [62]:
costing = "auto"
endpoint = "https://matrix.mapzen.com/sources_to_targets"

bigDF = pandas.DataFrame()

for n in range (0, breakup):
    colDF=pandas.DataFrame()
    
    for m in range (0,breakup):
        dictPath = {"sources":locations[n],
                    "targets":locations[m],
                    "costing":costing,
                    "id": filename}
        path = json.dumps(dictPath)
        params = {"json":path,"api_key":key}
        req = requests.get(endpoint, params=params)
        #print req.url
        jsonback = req.json()
        #locationsOut = jsonback.get("locations")
        rawMatrix = jsonback.get("sources_to_targets")
        
        #cols = list(range(1+n*50, 51+n*50)
        #rows = list(range(1+m*50, 51+m*50)
        cols = list(range(1+n*50, 1+n*50+len(locations[n])))
        rows = list(range(1+m*50, 1+m*50+len(locations[m])))
        mo = pandas.DataFrame(columns=cols, index=rows)
        
        for row in rawMatrix:
            for edge in row:
                D = edge['to_index']
                O = edge['from_index']
                distance = edge['distance']
                mo.iloc[D,O]= distance
                
        #build up matrix for this n to all m
        if m==0:
            colDF = mo
        else:
            colDF = colDF.append(mo)
            
    if n==0:
        bigDF = colDF
    else:
        bigDF = bigDF.merge(colDF, left_index=True, right_index=True, how='outer') 
            
        #mo.to_csv("dataOut/"+filename+"_"+str(n)+"_"+str(0)+".csv")
bigDF.to_csv("dataOut/"+filename+"_out.csv")

In [63]:
bigDF.tail()     

,1,2,3,4,5,6,7,8,9,10,...,66,67,68,69,70,71,72,73,74,75
71,46.223,81.813,47.997,40.437,44.953,36.919,52.107,52.276,61.328,55.043,...,15.463,15.424,18.088,32.245,18.267,0,9.901,10.529,7.386,24.97
72,52.547,79.885,39.32,38.509,43.025,34.039,43.429,49.396,52.65,45.183,...,21.117,13.373,13.635,23.567,13.814,8.678,0,10.172,7.68,12.197
73,56.984,61.096,55.694,48.134,52.65,44.211,53.601,59.568,62.822,55.355,...,17.872,10.128,10.561,24.374,10.74,11.755,10.172,0,3.78,22.369
74,57.628,61.74,41.116,47.453,51.969,41.719,51.109,57.076,60.33,52.863,...,18.702,10.958,11.22,25.033,11.399,7.386,7.68,3.78,0,19.877
75,65.11,85.177,51.361,43.801,48.317,25.649,35.039,41.006,42.786,36.793,...,29.653,25.629,20.726,22.377,18.071,21.107,12.429,22.601,20.109,0


## JUNK BELOW - ignore it

In [231]:
costing = "auto"
endpoint = "https://matrix.mapzen.com/sources_to_targets"

dictPath = {"sources":locations[1],
            "targets":locations[1],
            "costing":costing}
print dictPath
path = json.dumps(dictPath)
params = {"json":path,"api_key":key,"id": filename}
req = requests.get(endpoint, params=params)
#print req.url

{'sources': [{'lat': 51.523625399999993, 'lon': -0.13362449999999998}, {'lat': 51.524465399999997, 'lon': -0.13269420000000001}, {'lat': 51.524188000000002, 'lon': -0.13243099999999999}, {'lat': 51.518244000000003, 'lon': -0.12096510000000001}, {'lat': 51.518924669999997, 'lon': -0.13180378000000001}, {'lat': 51.519795000000002, 'lon': -0.1313291}, {'lat': 51.518234020000001, 'lon': -0.12843228000000001}, {'lat': 51.523468700000002, 'lon': -0.12731389999999998}], 'targets': [{'lat': 51.523625399999993, 'lon': -0.13362449999999998}, {'lat': 51.524465399999997, 'lon': -0.13269420000000001}, {'lat': 51.524188000000002, 'lon': -0.13243099999999999}, {'lat': 51.518244000000003, 'lon': -0.12096510000000001}, {'lat': 51.518924669999997, 'lon': -0.13180378000000001}, {'lat': 51.519795000000002, 'lon': -0.1313291}, {'lat': 51.518234020000001, 'lon': -0.12843228000000001}, {'lat': 51.523468700000002, 'lon': -0.12731389999999998}], 'costing': 'auto'}


In [232]:
costing = "auto"
endpoint = "https://matrix.mapzen.com/many_to_many"

loc = locations[1]

dictPath = {"locations":loc,
            "costing":costing}
print dictPath
path = json.dumps(dictPath)
params = {"json":path,"api_key":key,"id": filename}
req = requests.get(endpoint, params=params)
#print req.url

{'locations': [{'lat': 51.523625399999993, 'lon': -0.13362449999999998}, {'lat': 51.524465399999997, 'lon': -0.13269420000000001}, {'lat': 51.524188000000002, 'lon': -0.13243099999999999}, {'lat': 51.518244000000003, 'lon': -0.12096510000000001}, {'lat': 51.518924669999997, 'lon': -0.13180378000000001}, {'lat': 51.519795000000002, 'lon': -0.1313291}, {'lat': 51.518234020000001, 'lon': -0.12843228000000001}, {'lat': 51.523468700000002, 'lon': -0.12731389999999998}], 'costing': 'auto'}


## Generate multiple matrices and save each to a csv

In [48]:
costing = "auto"
endpoint = "https://matrix.mapzen.com/sources_to_targets"

for n in range (0,breakup):
    for m in range (0,breakup):
        dictPath = {"sources":locations[n],
                    "targets":locations[m],
                    "costing":costing,
                    "id": filename}
        path = json.dumps(dictPath)
        params = {"json":path,"api_key":key}
        req = requests.get(endpoint, params=params)
        #print req.url
        jsonback = req.json()
        #locationsOut = jsonback.get("locations")
        rawMatrix = jsonback.get("sources_to_targets")
        cols = list(range(1+n*50, 51+n*50))
        rows = list(range(1+m*50, 51+m*50))
        mo = pandas.DataFrame(columns=cols, index = rows)
        
        for row in rawMatrix:
            for edge in row:
                D = edge['to_index']
                O = edge['from_index']
                distance = edge['distance']
                mo.iloc[D,O]= distance
                
        mo.to_csv("dataOut/"+filename+"_"+str(n)+"_"+str(0)+".csv")
        